In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import torch
import torch.nn as nn

from plaid.vqvae import TransformerVQVAE
from plaid.utils import LatentToSequence, LatentToStructure

In [4]:
file = "/shared/amyxlu/plaid/checkpoints/2024-01-11T15-45-26/last.ckpt"
# ckpt = torch.load(file)
model = TransformerVQVAE.load_from_checkpoint(file)

In [9]:
from plaid.esmfold import esmfold_v1
esmfold = esmfold_v1()

In [10]:
device = torch.device("cuda:0")
latent_to_structure = LatentToStructure(device, esmfold)
latent_to_sequence = LatentToSequence(device)

In [12]:
from plaid.datasets import CATHShardedDataModule

dm = CATHShardedDataModule(
    shard_dir ="/shared/amyxlu/data/cath/shards",
    header_to_sequence_file="/shared/amyxlu/data/cath/sequences.pkl",
    seq_len=64,
    dtype="fp32",
)
dm.setup()
loader = dm.val_dataloader()
batch = next(iter(loader))

In [15]:
model = model.to(device)
model = model.eval().requires_grad_(False)

x = batch[0].to(device)
output = model(x)

In [21]:
x_hat = output[1]
torch.mean((x_hat - x) ** 2)

tensor(5212.0371, device='cuda:0')

In [27]:
sequences = [s[:64] for s in batch[-1]]
pred_pdb = latent_to_structure.to_structure(x_hat, sequences)

(Generating structure from latents..): 100%|██████████| 1/1 [11:18<00:00, 678.82s/it]
